In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

In [6]:
df = pd.read_csv('./data/insurance.csv')

In [7]:
df.sample(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
51,45,101.9,1.62,28.95,True,Jaipur,private_job,High
60,41,101.3,1.81,49.94,True,Jalandhar,unemployed,High
33,73,67.5,1.76,1.46,False,Mumbai,retired,Medium
31,39,51.1,1.83,11.77,True,Lucknow,private_job,Medium
79,39,55.3,1.56,30.00,True,Indore,government_job,Low


In [8]:
df_feat = df.copy()

In [9]:
# Feature 1: Calculate bmi and store to the dataframe
df_feat["bmi"] = df_feat["weight"]/(df_feat["height"]**2)

In [14]:
# Feature 2: Get group age and store to the dataframe
def age_group(age):
    if age < 25:
        return "young"
    elif age < 45:
        return "adult"
    elif age < 60:
        return "middle_aged"
    else:
        return "senior"

In [12]:
df_feat["age_group"] = df_feat["age"].apply(age_group)

In [15]:
# Feature 3: Get lifestyle risk and store to the dataframe
def lifestyle_risk(row):
    if row["smoker"] and row["bmi"] > 30:
        return "high"
    elif row["smoker"] and row["bmi"] > 27:
        return "medium"
    else:
        return "low"

In [16]:
df_feat["lifestyle_risk"] = df_feat.apply(lifestyle_risk, axis=1)

In [17]:

tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]

In [18]:
# Feature 4: City Tier
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

In [19]:
df_feat["city_tier"] = df_feat["city"].apply(city_tier)

In [20]:
df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_group', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)

,income_lpa,occupation,bmi,age_group,lifestyle_risk,city_tier,insurance_premium_category
9,43.07,business_owner,24.858833,middle_aged,low,1,Low
24,18.60,private_job,19.669038,middle_aged,low,2,Medium
95,19.64,business_owner,21.420747,adult,low,2,Low
35,43.28,private_job,20.762578,middle_aged,low,2,Medium
70,0.57,retired,36.694215,senior,low,2,High


In [21]:
# Select features and target
X = df_feat[["bmi", "age_group", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = df_feat["insurance_premium_category"]

In [22]:
# Define categorical and numeric features
categorical_features = ["age_group", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]

In [23]:
# Create column transformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)

In [24]:
# Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [25]:
# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [28]:
# Predict and evaluate
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.75

In [27]:
X_test.sample(5)

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
81,31.866055,adult,high,2,22.19,freelancer
80,34.350461,middle_aged,low,2,50.00,unemployed
32,31.495845,middle_aged,low,2,50.00,private_job
10,22.949982,adult,low,1,32.78,business_owner
56,42.414152,young,high,1,2.86,student


In [29]:
import pickle

# Save the trained pipeline using pickle
pickle_model_path = "model.pkl"
with open(pickle_model_path, "wb") as f:
    pickle.dump(pipeline, f)